## Baseball-specific analysis

D) List the 20 most dangerous pitchers derived from their career total statistics. Specifically, those with the highest career rates of hitting batters. (Join People & Pitching tables. Use the formula total “Batters Hit By Pitch” / total “Outs Pitched”)

In [3]:
import pandas as pd
import numpy as np
import doctest

Read the data files for pitches and player names

In [5]:
pitching = pd.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/Pitching.csv')

In [6]:
people = pd.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/People.csv')

Compute for each player, the number of Outs pitched and batters hit. Then create a column to hold the frequency with which batters are hit.

In [8]:
pitches_by_playerid = pitching.groupby('playerID').agg({'HBP':sum, 'IPouts':sum})
pitches_by_playerid.loc[:, 'batter_hit_freq'] = pitches_by_playerid.HBP/pitches_by_playerid.IPouts

The top 20 most dangerous pitchers can be identified by sorting based on frequency

In [9]:

def top_n(df:pd.DataFrame, column:str, n:int=20):
    """Return the top n rows of the dataframe df when sorted by column.
    
    df: dataframe whose data needs to be sorted.
    column: column on which the rows are to be ranked
    n: number of rows to be extracted
    
    >>> df = pd.DataFrame([['a',1], ['b', 2],['c', 3],['d', 4]], columns=['alpha', 'num'])
    >>> out = top_n(df, 'num', 2)
    >>> list(out.alpha.values)
    ['d', 'c']
    >>> list(out.num.values)
    [4, 3]
    """
    return df.sort_values(by=column, ascending=False).head(n)
doctest.testmod()

TestResults(failed=0, attempted=4)

We ignore players with '0' IPouts to get rid of misleading batting hit frequency

In [11]:
top_20_pitchers = top_n(pitches_by_playerid.loc[pitches_by_playerid.IPouts>0], 'batter_hit_freq', 20)

In [12]:
top_20_pitchers

,HBP,IPouts,batter_hit_freq
playerID,,,
sborzja01,2.0,2,1.000000
cathete01,1.0,1,1.000000
bleicje01,1.0,1,1.000000
brownpe01,1.0,1,1.000000
wilshte01,1.0,1,1.000000
youngjb01,3.0,6,0.500000
osikke01,3.0,6,0.500000
moorety01,1.0,2,0.500000
jonesga02,1.0,2,0.500000


join top_20_pitchers dataframe with people dataframe to identify the most dangerous pitchers

In [18]:
top_20_pitchers.merge(people, on='playerID')[['nameFirst', 'nameLast', 'HBP', 'IPouts', 'batter_hit_freq']]

,nameFirst,nameLast,HBP,IPouts,batter_hit_freq
0,Jay,Sborz,2.0,2,1.000000
1,Ted,Cather,1.0,1,1.000000
2,Jeremy,Bleich,1.0,1,1.000000
3,Pete,Browning,1.0,1,1.000000
4,Terry,Wilshusen,1.0,1,1.000000
5,J. B.,Young,3.0,6,0.500000
6,Keith,Osik,3.0,6,0.500000
7,Tyler,Moore,1.0,2,0.500000
8,Garrett,Jones,1.0,2,0.500000
9,George,Craig,2.0,5,0.400000


In [ ]:
# Part 2 cross-database integration analysis

## Question 1

In [28]:
import pandas as pd
import numpy as np

In [29]:
people = pd.read_csv('baseballdatabank-2019.2/baseballdatabank-2019.2/core/People.csv')

In [30]:
people_names = people[['playerID', 'birthYear', 'nameGiven']].copy()

In [31]:
people_names.nameGiven.str.split(' ', expand=True).loc[:,0]

0           David
1           Henry
2          Tommie
3          Donald
4          Fausto
5        Fernando
6            John
7          Edward
8            Bert
9         Charles
10        Leander
11          Harry
12        William
13        Jeffrey
14          James
15           Kurt
16       Lawrence
17            Ody
18           Paul
19         Albert
20        Francis
21       Reginald
22        William
23        Michael
24       Talmadge
25       Theodore
26         Virgil
27       Clifford
28          Harry
29          Shawn
           ...   
19587      Jordan
19588         Roy
19589        Ryan
19590     Charles
19591      Walter
19592       Frank
19593         Guy
19594       James
19595     William
19596        Alan
19597      Marion
19598     Richard
19599       Barry
19600     William
19601      Edward
19602    Benjamin
19603       Peter
19604      Samuel
19605      Edward
19606     William
19607         Jon
19608       Julio
19609        Joel
19610     Michael
19611     

In [32]:
people_names.loc[:,'first_names']=people_names.nameGiven.str.split(' ', expand=True).loc[:,0]

In [33]:
people_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19617 entries, 0 to 19616
Data columns (total 4 columns):
playerID       19617 non-null object
birthYear      19497 non-null float64
nameGiven      19580 non-null object
first_names    19580 non-null object
dtypes: float64(1), object(3)
memory usage: 613.1+ KB


In [34]:
people_names.dropna(how = 'any', subset = ['birthYear', 'nameGiven'], inplace=True)

In [35]:
people_names.birthYear = people_names.birthYear.astype(int)

In [36]:
people_names.birthYear.min(), people_names.birthYear.max()

(1820, 1998)

In [38]:
def process_name_file(year, gender='M'):
    file = 'names/yob{}.txt'.format(year)
    df = pd.read_csv(file, header=None, names=['Name', 'Sex', 'noccur'])
    df = df.loc[df.Sex == gender, ['Name', 'noccur']].copy()
    df.loc[:,str(year)]=df.noccur/df.noccur.sum()
    return df.drop(['noccur'], axis=1)
    

In [37]:
def process_ssn(min_year, max_year, path='names'):
    names_accum = process_name_file(min_year)
    for year in range(min_year+1, max_year+1):        
        df = process_name_file(year, gender='M')
        names_accum = names_accum.merge(df, on='Name', how='outer')
    return names_accum

In [39]:
names_count = (people_names.groupby(['first_names', 'birthYear'])
               .first_names
               .count()
               .rename('name_count')
              )


In [40]:
names_count = names_count.reset_index()


In [41]:
players_born = names_count.groupby('birthYear').name_count.sum().to_frame().rename(columns={'name_count':'nbirths'}).reset_index()

In [42]:
players_born.head()

,birthYear,nbirths
0,1820,1
1,1824,1
2,1832,3
3,1835,1
4,1836,1


In [43]:
names_count = names_count.merge(players_born, on='birthYear')

In [44]:
names_count.head()

,first_names,birthYear,name_count,nbirths
0,A.,1869,1,105
1,Alexander,1869,3,105
2,Alfred,1869,2,105
3,Algernon,1869,1,105
4,Anthony,1869,1,105


In [45]:
names_count.loc[:,'frequency']= names_count.name_count/names_count.nbirths

In [47]:
freq = process_ssn(1880, 1998)

In [48]:
frequency = freq.melt(id_vars = ['Name'], var_name='birthYear')

In [49]:
frequency.birthYear=frequency.birthYear.astype(int)

In [50]:
merged = names_count.merge(frequency, left_on=['birthYear','first_names'], right_on=['birthYear', 'Name'])

In [51]:
merged.loc[merged['frequency']>merged['value']]


,first_names,birthYear,name_count,nbirths,frequency,Name,value
0,Aaron,1896,1,135,0.007407,Aaron,0.000577
1,Alan,1896,1,135,0.007407,Alan,0.000109
3,Alexander,1896,1,135,0.007407,Alexander,0.001940
4,Alfred,1896,1,135,0.007407,Alfred,0.004408
5,Allen,1896,1,135,0.007407,Allen,0.001832
6,Andrew,1896,1,135,0.007407,Andrew,0.004734
9,Bartholomew,1896,1,135,0.007407,Bartholomew,0.000067
10,Bennett,1896,1,135,0.007407,Bennett,0.000151
11,Bernard,1896,2,135,0.014815,Bernard,0.002016
12,Byron,1896,1,135,0.007407,Byron,0.000493
